In [2]:
import http.client
import json
import pandas as pd
from pymongo import MongoClient

In [3]:


def fetch_and_save_api_data(file_name, api_url, endpoint, headers):
    """Fetch data from an API endpoint and save it to a JSON file."""
    conn = http.client.HTTPSConnection(api_url)
    try:
        conn.request("GET", endpoint, headers=headers)
        response = conn.getresponse()
        data = json.loads(response.read().decode("utf-8"))
        with open(file_name, 'w') as f:
            json.dump(data, f, indent=4)
        print(f"Data successfully saved to {file_name}")
        return file_name
    except Exception as e:
        print(f"Failed to fetch and save data: {e}")
        return None

def upload_data_to_mongodb(file_name, db_name, collection_name, file_type='json'):
    """Upload data from a file directly to a MongoDB collection."""
    try:
        client = MongoClient('mongodb://localhost:27017/')
        db = client[db_name]
        collection = db[collection_name]
        
        # Read data based on file type
        if file_type == 'json':
            with open(file_name, 'r') as file:
                data = json.load(file)
        elif file_type == 'csv':
            df = pd.read_csv(file_name)
            data = df.to_dict('records')
        else:
            raise ValueError("Unsupported file type. Use 'json' or 'csv'.")

        # Insert data into MongoDB
        if isinstance(data, list):
            collection.insert_many(data)
        else:
            collection.insert_one(data)
        print(f"Data successfully uploaded to MongoDB in the '{collection_name}' collection")
    except Exception as e:
        print(f"Failed to upload data to MongoDB from file {file_name}: {e}")

# Example of how to use the function correctly



In [7]:
# Block for DrugsOverDose Collection
headers = {
    'X-RapidAPI-Key': "02ffe3eedcmsh472575185a8e638p1f6d68jsneffb8f95f5c7",
    'X-RapidAPI-Host': "drug-overdose-death.p.rapidapi.com"
}
drug_file = fetch_and_save_api_data('drug_overdose.json', 'drug-overdose-death.p.rapidapi.com', "/records?orderBy=dataListIndex_asc&limit=500", headers)
if drug_file:
    upload_data_to_mongodb(drug_file, 'HealthCare', 'DrugsOverDose')

Failed to fetch and save data: Expecting value: line 1 column 1 (char 0)


In [4]:
.

Data successfully uploaded to MongoDB in the 'DeathRelatedDrug' collection


In [6]:
# Block for AppointmentDetails Collection

upload_data_to_mongodb('appointment_details.csv', 'HealthCare', 'AppointmentDetails', 'csv')

Data successfully uploaded to MongoDB in the 'AppointmentDetails' collection
